In [1]:
import OffsetArrays
using Plots
using Revise
using StatsBase
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using Distributions
srcpath = "./src"
include("$srcpath/bp.jl") 
include("$srcpath/bp_stab.jl") 
include("$srcpath/post_infer.jl") 
include("$srcpath/observables.jl")

avg_ninf (generic function with 1 method)

In [2]:
λp = 1.0# planted infection rate
λi = λp #inferred infection rate

T = 8 # discrete time: number of time-steps

γp = 0.02 #inferred autoinfection probability
γi = γp

N = 5000; #population size
@show 2/sqrt(N)

dilution = 0.0 #dilution of observations. dil=0 means everybody observed once
fr = 0.0; #noise in the observation. 

degree_dist = Dirac(3) #the distribution of the degree in the graph. Dirac means random regular.

2 / sqrt(N) = 0.0282842712474619


Dirac{Int64}(value=3)

In [3]:
M_zero = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);
@time sweep!(M_zero)
F_zero,it_zero = pop_dynamics(M_zero, tot_iterations = 40, stop_at_convergence=false);

  2.583526 seconds (25.21 M allocations: 1.620 GiB, 8.91% gc time, 63.88% compilation time)
#1.iter 2.F
0	-0.24312748230257056
1	0.17782619285265536
2	0.2542206894570656
3	0.2546441249172698
4	0.25462030535436786
5	0.25461268328700826
6	0.25461253486898056
7	0.25461255564983293
8	0.25461255828198776
9	0.25463487249388417
10	0.2545456162679577
11	0.2546348724835523
12	0.25463487259483225
13	0.2546125582902197
14	0.2546125582902197
15	0.2546125582902197
16	0.2546125582902197
17	0.2546125582902197
18	0.2546125582902197
19	0.2546125582902197
20	0.25465718667688636
21	0.25465718667688486
22	0.25458128350870246
23	0.2546125582902197
24	0.2546125582902197
25	0.2546125582902197
26	0.2546125582902197
27	0.2546125582902197
28	0.25459024431944416
29	0.2546125582902197
30	0.25461255840149677
31	0.2546125582902197
32	0.2546125582902197
33	0.2546125582902197
34	0.2546125582902197
35	0.2546348724835523
36	0.2520832384893496
37	0.25465880380308953
38	0.23890560887100581
39	0.288678640430799


In [4]:
M = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);
#@time sweep_stab!(M, 3, 4)
pop_dynamics_stab(M, tot_iterations = 40);

#1.iter 2.F 3.Δ
1	0.13662919672794263	-1
2	-0.015264797493006682	-1
3	0.07900967052428938	-1
4	0.19458667254942277	-1
5	0.24230919861730313	-1
6	0.25208008492090195	-1
7	0.2533052702011354	-1
8	0.2537681838507394	-1
9	0.25442439766765457	-1
10	0.254603196403817	-1
11	0.25463289516388116	-1
12	0.25462797049370367	-1
13	0.25462041073609903	-1
14	0.25461596822262705	-1
15	0.2546139349774297	-1
16	0.2546577273139452	-1
17	0.2546127271187719	-1
18	0.2546125873152191	-1
19	0.25461254870734756	-1
20	0.2546125465067231	-1
21	0.2546125511993108	1.314530090536297e-5
22	0.2546125548998531	1.8195305841934255e-5
23	0.25461255686983153	1.4352018922096804e-5
24	0.25461255776572517	6.922672165500064e-6
25	0.2546125581220644	3.0649779519153098e-6
26	0.25461255825219287	1.777379537746261e-6
27	0.25461255829120105	8.791582475565668e-7
28	0.25470181495884925	3.3971061825257187e-7
29	0.254545616048129	9.192299354822045e-8
30	0.25461255829128226	1.7121990181527488e-8
31	0.25461255828948887	8.021733892392766

In [13]:
M.μ[:,1,:,1,rand(1:N)]

10×10 OffsetArray(::Matrix{Float64}, 0:9, 0:9) with eltype Float64 with indices 0:9×0:9:
 0.0  0.0  0.0  0.0  0.0  0.00843925   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.00843904   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.00827014   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.00794255   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.00732584   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.00623238   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.00451073   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.00236279   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.000648013  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  4.84921e-5   0.0  0.0  0.0  0.0

In [6]:
γp

0.02

In [7]:
γp=0.005
M2 = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);
pop_dynamics_stab(M2, tot_iterations = 40);

#1.iter 2.F 3.Δ
1	0.14367943979774656	-1
2	-0.014908319370215995	-1
3	0.08239066125764978	-1
4	0.19106214142749203	-1
5	0.2427676776377946	-1
6	0.19741235630808768	-1
7	-0.3855483071524452	-1
8	-3.3810481668042596	-1
9	-9.912559109449004	-1
10	-9.857429517421938	-1
11	0.053514848297814215	-1
12	4.744791847829978	-1
13	7.978887002230121	-1
14	4.879976672461474	-1
15	0.6708313312084836	-1
16	2.2347634552643822	-1
17	5.782658655428014	-1
18	4.850773206346791	-1
19	1.5290409467657655	-1
20	-0.2190097088298411	-1
21	-0.5475555081189556	4.095812164315624e23
22	0.5325166696020649	3.902130135404515e42
23	1.8388969934719812	9.072438494528649e89
24	1.6337726684324403	3.893231503040609e91
25	1.4769046659519138	8.251666985782504e89
26	0.9907277709918281	1.0552804794158432e78
27	0.6282522002159581	2.2791886666825025e97
28	0.5611509121306572	2.800443467476318e118
29	0.5845264819038104	Inf
30	0.6435890224618239	9.061442754539953e125
31	0.5706458805717214	1.3204770558418043e127
32	0.5456928115206824	I

In [27]:
M2.mom1μ[:,1,:,1,rand(1:N)]

10×10 OffsetArray(::Matrix{Float64}, 0:9, 0:9) with eltype Float64 with indices 0:9×0:9:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0         0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0         0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0         0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.79026e72  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.79026e72  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.79026e72  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.79026e72  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.79026e72  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.79026e72  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0         0.0

In [43]:
σ0=0.00001

1.0e-5

In [44]:
M2.mom1μ

10×2×10×3×5000 OffsetArray(::Array{Float64, 5}, 0:9, 0:1, 0:9, 0:2, 1:5000) with eltype Float64 with indices 0:9×0:1×0:9×0:2×1:5000:
[:, :, 0, 0, 1] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 1, 0, 1] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 2, 0, 1] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 3, 0, 1] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 4, 0, 1] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 5, 0, 1] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 6, 0, 1] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 7, 0, 1] =
 0.0  0.0
 0.0  

In [46]:
OffsetArray(rand(-1:2:1, T+2, 2, T+2, 3, N).*σ0, 0:T+1, 0:1, 0:T+1, 0:2, 1:N).*(M2.μ)

10×2×10×3×5000 OffsetArray(::Array{Float64, 5}, 0:9, 0:1, 0:9, 0:2, 1:5000) with eltype Float64 with indices 0:9×0:1×0:9×0:2×1:5000:
[:, :, 0, 0, 1] =
  0.0  -0.0
  0.0  -0.0
  0.0  -0.0
 -0.0  -0.0
  0.0  -0.0
 -0.0   0.0
 -0.0   0.0
 -0.0  -0.0
  0.0  -0.0
 -0.0   0.0

[:, :, 1, 0, 1] =
 -0.0  -0.0
  0.0  -0.0
  0.0  -0.0
 -0.0  -0.0
  0.0   0.0
 -0.0   0.0
 -0.0  -0.0
 -0.0   0.0
  0.0   0.0
  0.0  -0.0

[:, :, 2, 0, 1] =
 -0.0   0.0
 -0.0  -0.0
 -0.0   0.0
  0.0   0.0
 -0.0  -0.0
  0.0   0.0
 -0.0   0.0
 -0.0   0.0
  0.0  -0.0
 -0.0   0.0

[:, :, 3, 0, 1] =
  0.0   0.0
  0.0   0.0
  0.0   0.0
 -0.0   0.0
  0.0  -0.0
 -0.0   0.0
 -0.0   0.0
 -0.0   0.0
 -0.0   0.0
  0.0   0.0

[:, :, 4, 0, 1] =
  0.0   0.0
 -0.0   0.0
  0.0   0.0
 -0.0   0.0
 -0.0   0.0
 -0.0  -0.0
 -0.0  -0.0
  0.0   0.0
 -0.0   0.0
 -0.0   0.0

[:, :, 5, 0, 1] =
 -0.0   0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
  0.0   0.0
  0.0  -0.0
  0.0  -0.0
 -0.0   0.0
  0.0  -0.0
 -0.0  -0.0

[:, :, 6, 0, 1] =
 -0.0  -0.0
 -0

In [29]:
M.μ[:,1,:,1,rand(1:N)]

10×10 OffsetArray(::Matrix{Float64}, 0:9, 0:9) with eltype Float64 with indices 0:9×0:9:
 0.0  0.0  0.0  0.0  0.00843925   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.00843904   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.00827014   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.00794255   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.00732584   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.00623238   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.00451073   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.00236279   0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.000648013  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  4.84921e-5   0.0  0.0  0.0  0.0  0.0

In [ ]:
M3 = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [ ]:
marg_zero = M_zero.belief |> real;
marg2D_zero = reshape(sum(marg_zero,dims=3) ./ N, T+2,T+2)
ensAUC_zero = avgAUC(marg_zero,M_zero.obs_list,count_obs=true); #set false to see the AUC only on NON observed indiv.

In [ ]:
marg = M.belief |> real;
marg2D = reshape(sum(marg,dims=3) ./ N, T+2,T+2)
ensAUC = avgAUC(marg,M.obs_list,count_obs=true); #set false to see the AUC only on NON observed indiv.

In [ ]:
pheat = heatmap((marg2D_zero),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred",title="Average 2D marginal")
pheat_stab = heatmap(marg2D,c=[:white,:blue,:black],xlabel="planted",ylabel="inferred")
plot(pheat,pheat_stab)